# Preparation

## Authentication

Before requests to orchestration can be issued, we need to provide authentication details to the SDK. This can be done either via a configuration file or via the environment. Make sure to check out the [SAP generative AI hub SDK project description](https://pypi.org/project/generative-ai-hub-sdk/) for more details. Below you will find an example for authenticating via environment variables using this very notebook.

> **WARNING:**
> Below code should never be used in production scenarios and is only for the purpose of illustrating which environment variables to use!
> Credentials should never be defined in code!

In [1]:
import json

with open('../aicore_proxy_participant_credentials_ai180p104road.json', 'r') as file:
    conn_data = json.load(file)

In [ ]:
conn_data

In [3]:
import os

os.environ["AICORE_AUTH_URL"] = conn_data["python"]["AICORE_AUTH_URL"]

In [ ]:
os.environ["AICORE_AUTH_URL"]

In [5]:
import os

os.environ["AICORE_AUTH_URL"] = conn_data["python"]["AICORE_AUTH_URL"]
os.environ["AICORE_BASE_URL"] = conn_data["python"]["AICORE_BASE_URL"]
os.environ["AICORE_CLIENT_ID"] = conn_data["python"]["AICORE_CLIENT_ID"]
os.environ["AICORE_CLIENT_SECRET"] = conn_data["python"]["AICORE_CLIENT_SECRET"]
os.environ["AICORE_ORCHESTRATION_DEPLOYMENT_URL"] = conn_data["python"]["AICORE_ORCHESTRATION_DEPLOYMENT_URL"]
os.environ["AICORE_RESOURCE_GROUP"] = conn_data["python"]["AICORE_RESOURCE_GROUP"]

You will need to authenticate in every exercise.

# Orchestration Chatbot

We will now combine templating and content filtering in a chatbot. Additionally we will make use of the orchestration services history capability.

We start with building the chatbot class. This class will be responsible for providing the template for the user query and managing the history. Note that the `chat` method uses the `history` parameter to send along the history to the orchestration service.

In [17]:
from typing import List

from gen_ai_hub.orchestration.models.message import Message, SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.service import OrchestrationService


class OrchestrationChatbot:
    def __init__(self, orchestration_service: OrchestrationService):
        self.service = orchestration_service
        self.service.config.template = Template(
            messages=[
                SystemMessage("You are a helpful chatbot assistant speaking like a bloody pirate."),
                UserMessage("{{?user_query}}"),
            ],
        )
        self.history: List[Message] = []

    def chat(self, user_input):
        response = self.service.run(
            template_values=[
                TemplateValue(name="user_query", value=user_input),
            ],
            history=self.history,
        )

        message = response.orchestration_result.choices[0].message

        self.history = response.module_results.templating
        self.history.append(message)

        return message.content

Next up we will create an orchestration configuration. Next to the model parameter we also pass a content filter for input and output to ensure a nice conversation.

In [ ]:
from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.models.content_filter import AzureContentFilter
from gen_ai_hub.orchestration.models.llm import LLM

content_filter = AzureContentFilter(
    hate=0,
    sexual=0,
    self_harm=0,
    violence=0,
)
config = OrchestrationConfig(
    llm=LLM(name="gemini-1.5-flash"),
    template=None,
    input_filters=[content_filter],
    output_filters=[content_filter],
)

orchestration_service = OrchestrationService(
    api_url=os.environ["AICORE_ORCHESTRATION_DEPLOYMENT_URL"],
    config=config,
)

Lets chat with our bot and test if it is able to recall a conversation!

In [18]:
bot = OrchestrationChatbot(orchestration_service=orchestration_service)

In [ ]:
print(bot.chat("Hello, how are you?"))

In [ ]:
print(bot.chat("What's the weather like today?"))

In [ ]:
print(bot.chat("Can you remember what I first asked you?"))

# Summary

In this exercise you got hands-on experience on how to use orchestration in complex scenarios. This exercise concludes the SAP generative AI hub SDK (Python) track. Feel free to further explore the orchestration capabilities using this SDK or checkout the [SAP Cloud SDK for AI (JavaScript)](https://github.com/SAP-samples/teched2024-AI180/blob/main/exercises/javascript/README.md) track.